In [1]:
import pandas as pd
import os
import glob


### Important information, read below!


For each cancer type as well as the pan-cancer cohort we run a LOH, amplification and deep-deletion randomization script to detect genes under positive selection in both Hartwig and PCAWG samples. This analysis is performed at different genomic scales: hihgly-focal (<3Mb), focal (<75% chr. arm) and non-focal. 

This analyis is performed in two steps: 

1. Randomization per cancer type, cohort, CNV-type and scale. The script that performs the randomizations can be found at scripts/positive_selection/cnv/positive_selection_CNV_shuffle.py, whereas the script to submit all randomizations across cancer types is scripts/positive_selection/cnv/pipeline_pos_selection_cnv.py. Further details can be found in the source code of the scripts. 

2. Mapping of the randomization into the genes of interest. The output of these scripts is further parsed by CNV-type  (LOH, amp, deepdel) using the scripts scripts/positive_selection/cnv/process_output_LOH.py (process_output_AMP.py & process_output_DEL.py respectively). This parsing maps the genes of interest (HLA-I genes for LOH, and rest of immune-related genes for AMP&DEL) into the randomizations performed in the first step. Moreover

The output from the 2. steps provides, for the genes of interest, the significance of the performed analysis as well as the figures of the randomization used in the manuscript. 

### Read output for CNV in Hartwig

In [2]:
l=[]
for filein in glob.glob("/home/fran/Documents/cuppen/HPC/tunnel/cuppen/projects/P0020_genetics_immune_escape/large_scale_primary_met/processed/hmf/positive_selection/LOH/ignore_*/plots/*.tsv.gz"):
    df=pd.read_csv(filein,sep="\t")
    #df=df[df["q_value_ana_global"]<.1]
    type_alt,focal=os.path.basename(filein).split(".")[0].split("_")[1:3]
    df["type_alteration"] = type_alt
    df["focal"] = focal
    if df.shape[0] >0:

        l.append(df)
df_total = pd.concat(l)
df_total["cancer_type"] = df_total.apply(lambda r: r["ttype"].replace("__"," "),axis=1)
df_total[(df_total["type_alteration"]!="loh")|((df_total["type_alteration"]=="loh")&(df_total["gene"].str.contains("HLA")))].drop(["ttype","pvalue_emp_locla"],axis=1).sort_values("q_value_ana_global").to_csv("../results/data/raw_results_cnv_hartwig.tsv.gz",sep="\t",index=False,compression="gzip")

### Only significant

In [3]:
l=[]
for filein in glob.glob("/home/fran/Documents/cuppen/HPC/tunnel/cuppen/projects/P0020_genetics_immune_escape/large_scale_primary_met/processed/hmf/positive_selection/LOH/ignore_*/plots/*.tsv.gz"):
    df=pd.read_csv(filein,sep="\t")
    df=df[df["q_value_ana_global"]<.1]
    type_alt,focal=os.path.basename(filein).split(".")[0].split("_")[1:3]
    df["type_alteration"] = type_alt
    df["focal"] = focal
    if df.shape[0] >0:

        l.append(df)
    

In [4]:
df_total[(df_total["q_value_ana_global"]<0.1)&(df_total["type_alteration"]=="loh")&(df_total["gene"]=="HLA-A")].sort_values("q_value_ana_global")

,ttype,gene,region,p_value_ana_global,q_value_ana_global,odds_ratio_global,n_obs,n_sim,mean_global,type_alteration,focal,pvalue_emp_locla,cancer_type
51,Cervix__carcinoma,HLA-A,chr6_29900001-30000000,0.001671,0.036239,2.650135,14,6.3,5.282750,loh,focal,NaN,Cervix carcinoma
36,Pancreas__neuroendocrine,HLA-A,chr6_29900001-30000000,0.005864,0.054733,2.048647,18,9.0,8.786289,loh,nonfocal,NaN,Pancreas neuroendocrine
60,Colorectum__carcinoma,HLA-A,chr6_29900001-30000000,0.004563,0.076115,2.174080,17,6.7,7.819398,loh,hfocal,NaN,Colorectum carcinoma


In [5]:
import numpy as np
def concat(grp):
    return ",".join(set(grp))
result_hartwig=df_total[(df_total["q_value_ana_global"]<0.1)&(df_total["n_obs"]>2)&((df_total["type_alteration"]=="loh")&(df_total["gene"].str.contains("HLA"))|((df_total["type_alteration"]!="loh")))&(df_total["odds_ratio_global"]>1.)
# Manual inspection of HLA-A deepdeletions in esophagus reveal a potential artifact caused by the high number of LINE insertions in this region. See methods of manuscript for more details
         &(~((df_total["ttype"].isin(["pancancer","Esophagus__cancer"]))&(df_total["gene"]=="HLA-A")&(df_total["type_alteration"]=="deepdel")))].groupby(["ttype","gene","type_alteration"]).agg({"region":concat,"q_value_ana_global":np.nanmin,"odds_ratio_global":np.nanmax,"n_obs":np.nanmax,"n_sim":np.nanmax,"mean_global":np.nanmax,"focal":concat})
result_hartwig.to_csv("../results/data/CNV_positive_selection_HMF.tsv",sep="\t")

In [6]:
result_hartwig

region  \
ttype                            gene   type_alteration                                                      
Breast__cancer                   JAK2   deepdel                                       chr9_5000001-5100000   
Cervix__carcinoma                HLA-A  loh                                         chr6_29900001-30000000   
                                 HLA-B  loh                                         chr6_31300001-31400000   
                                 HLA-C  loh                                         chr6_31200001-31300000   
Colorectum__carcinoma            HLA-A  loh                                         chr6_29900001-30000000   
                                 IRF2   deepdel                                   chr4_185300001-185400000   
Diffuse__large__B-cell__lymphoma B2M    deepdel                                    chr15_45000001-45100000   
Non__small__cell__lung__cancer   SETDB1 amp              chr1_150800001-150900000,chr1_150900001-151000000   
Ovarian__cancer                  B2M    deepdel                                    chr15_45000001-45100000   
                                 JAK2   deepdel                                       chr9_4900001-5000000   
                                 SETDB1 amp              chr1_150800001-150900000,chr1_150900001-151000000   
Pancreas__neuroendocrine         HLA-A  loh                                         chr6_29900001-30000000   
                                 HLA-B  loh                                         chr6_31300001-31400000   
                                 HLA-C  loh                                         chr6_31200001-31300000   
Prostate__carcinoma              JAK1   deepdel          chr1_65400001-65500000,chr1_65200001-65300000,...   
Skin__melanoma                   B2M    deepdel                                    chr15_45000001-45100000   
Urothelial__cancer               SETDB1 amp                                       chr1_150800001-150900000   
pancancer                        B2M    deepdel                                    chr15_45000001-45100000   
                                 IRF2   deepdel                                   chr4_185300001-185400000   
                                 JAK1   deepdel          chr1_65400001-65500000,chr1_65200001-65300000,...   
                                 JAK2   deepdel          chr9_4900001-5000000,chr9_5000001-5100000,chr9...   
                                 SETDB1 amp              chr1_150800001-150900000,chr1_150900001-151000000   

                                                         q_value_ana_global  \
ttype                            gene   type_alteration                       
Breast__cancer                   JAK2   deepdel                1.638807e-03   
Cervix__carcinoma                HLA-A  loh                    3.623937e-02   
                                 HLA-B  loh                    1.278601e-02   
                                 HLA-C  loh                    1.278601e-02   
Colorectum__carcinoma            HLA-A  loh                    7.611535e-02   
                                 IRF2   deepdel                9.155631e-09   
Diffuse__large__B-cell__lymphoma B2M    deepdel                2.228786e-04   
Non__small__cell__lung__cancer   SETDB1 amp                    4.468130e-07   
Ovarian__cancer                  B2M    deepdel                6.481895e-03   
                                 JAK2   deepdel                9.498565e-02   
                                 SETDB1 amp                    9.687135e-05   
Pancreas__neuroendocrine         HLA-A  loh                    5.473274e-02   
                                 HLA-B  loh                    5.473274e-02   
                                 HLA-C  loh                    5.473274e-02   
Prostate__carcinoma              JAK1   deepdel                8.233580e-09   
Skin__melanoma                   B2M    deepdel                2.338527e-03   
Urothelial__cancer               SETD

In [11]:
np.log2(2.174080)

1.1204050284423892

### Read output for PCAWG

In [7]:
l=[]
for filein in glob.glob("/home/fran/Documents/cuppen/HPC/tunnel/cuppen/projects/P0020_genetics_immune_escape/large_scale_primary_met/processed/pcawg/positive_selection/LOH/ignore_*/plots/*.tsv.gz"):
    df=pd.read_csv(filein,sep="\t")
    #df=df[df["q_value_ana_global"]<.1]
    type_alt,focal=os.path.basename(filein).split(".")[0].split("_")[1:3]
    df["type_alteration"] = type_alt
    df["focal"] = focal
    if df.shape[0] >0:

        l.append(df)
df_total = pd.concat(l)
df_total["cancer_type"] = df_total.apply(lambda r: r["ttype"].replace("__"," "),axis=1)
df_total[(df_total["type_alteration"]!="loh")|((df_total["type_alteration"]=="loh")&(df_total["gene"].str.contains("HLA")))].drop(["ttype","pvalue_emp_locla"],axis=1).sort_values("q_value_ana_global").to_csv("../results/data/raw_results_cnv_pcawg.tsv.gz",sep="\t",index=False,compression="gzip")

In [8]:
result_pcawg=df_total[(df_total["q_value_ana_global"]<0.1)&(df_total["n_obs"]>2)&((df_total["type_alteration"]=="loh")&(df_total["gene"].str.contains("HLA"))|((df_total["type_alteration"]!="loh")))&(df_total["odds_ratio_global"]>1.)
# Manual inspection of HLA-A deepdeletions in esophagus reveal a potential artifact caused by the high number of LINE insertions in this region. See methods of manuscript for more details
         &(~((df_total["ttype"].isin(["pancancer","Esophagus__cancer"]))&(df_total["gene"]=="HLA-A")&(df_total["type_alteration"]=="deepdel")))].groupby(["ttype","gene","type_alteration"]).agg({"region":concat,"q_value_ana_global":np.nanmin,"odds_ratio_global":np.nanmax,"n_obs":np.nanmax,"n_sim":np.nanmax,"mean_global":np.nanmax,"focal":concat})
result_pcawg.to_csv("../results/data/CNV_positive_selection_PCAWG.tsv",sep="\t")

In [9]:
result_pcawg

region  \
ttype                            gene   type_alteration                                                      
Breast__cancer                   SETDB1 amp              chr1_150800001-150900000,chr1_150900001-151000000   
Diffuse__large__B-cell__lymphoma CD58   deepdel                                   chr1_117000001-117100000   
                                 HLA-A  loh                                         chr6_29900001-30000000   
                                 HLA-B  loh                                         chr6_31300001-31400000   
                                 HLA-C  loh                                         chr6_31200001-31300000   
Hepatocellular__carcinoma        IRF2   deepdel                                   chr4_185300001-185400000   
                                 SETDB1 amp              chr1_150800001-150900000,chr1_150900001-151000000   
Kidney__chromophobe__cancer      HLA-A  loh                                         chr6_29900001-30000000   
                                 HLA-B  loh                                         chr6_31300001-31400000   
                                 HLA-C  loh                                         chr6_31200001-31300000   
Pancreas__carcinoma              IRF2   deepdel                                   chr4_185300001-185400000   
Pancreas__neuroendocrine         HLA-A  loh                                         chr6_29900001-30000000   
                                 HLA-B  loh                                         chr6_31300001-31400000   
                                 HLA-C  loh                                         chr6_31200001-31300000   
Prostate__carcinoma              JAK1   deepdel                                     chr1_65300001-65400000   
Skin__melanoma                   CALR   deepdel                                    chr19_13000001-13100000   
pancancer                        IRF2   deepdel                                   chr4_185300001-185400000   
                                 JAK1   deepdel                                     chr1_65300001-65400000   
                                 JAK2   deepdel                                       chr9_5100001-5200000   
                                 SETDB1 amp              chr1_150800001-150900000,chr1_150900001-151000000   

                                                         q_value_ana_global  \
ttype                            gene   type_alteration                       
Breast__cancer                   SETDB1 amp                        0.000794   
Diffuse__large__B-cell__lymphoma CD58   deepdel                    0.004178   
                                 HLA-A  loh                        0.013238   
                                 HLA-B  loh                        0.005445   
                                 HLA-C  loh                        0.005445   
Hepatocellular__carcinoma        IRF2   deepdel                    0.000174   
                                 SETDB1 amp                        0.045483   
Kidney__chromophobe__cancer      HLA-A  loh                        0.000100   
                                 HLA-B  loh                        0.000100   
                                 HLA-C  loh                        0.000100   
Pancreas__carcinoma              IRF2   deepdel                    0.067895   
Pancreas__neuroendocrine         HLA-A  loh                        0.000418   
                                 HLA-B  loh                        0.000418   
                                 HLA-C  loh                        0.000418   
Prostate__carcinoma              JAK1   deepdel                    0.097553   
Skin__melanoma                   CALR   deepdel                    0.051430   
pancancer                        IRF2   deepdel                    0.024270   
                                 JAK1   deepdel                    0.087733   
                                 JAK2   deepdel                    0.087733   
                    

In [12]:
np.log2(2.596173)

1.3763865227538876

In [13]:
df_meta

NameError: name 'df_meta' is not defined